In [1]:
# P Naidoo

In [2]:
import os
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
path_to_videos = 'c:\Data\echonet\EchoNet-Dynamic\Videos'
volumetracingsCSV = 'c:\Data\echonet\EchoNet-Dynamic\VolumeTracings.csv'
outputvolumetracingsFramesPath = 'volumetracingsFrames'
outputIMAGESwithExpertTracingsPath = 'volumetracingsIMAGESwithExpertTracings'
outputApproximatedLVsPath = 'volumetracingsApproximatedLV'
outputApproximatedLVBinarPath = 'volumetracingsApproximatedLVBinary'
outputApproximatedLVCoordinatesPath = 'volumetracingsApproximatedLVCoordinates'

if not os.path.exists(outputvolumetracingsFramesPath):
    os.makedirs(outputvolumetracingsFramesPath)
if not os.path.exists(outputIMAGESwithExpertTracingsPath):
    os.makedirs(outputIMAGESwithExpertTracingsPath)
if not os.path.exists(outputApproximatedLVsPath):
    os.makedirs(outputApproximatedLVsPath)
if not os.path.exists(outputApproximatedLVBinarPath):
    os.makedirs(outputApproximatedLVBinarPath)
if not os.path.exists(outputApproximatedLVCoordinatesPath):
    os.makedirs(outputApproximatedLVCoordinatesPath)

In [4]:
#Group info into data structures
my_data = {}

with open(volumetracingsCSV) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in csv_reader:
        #line_count = 0 is column names
        #row is of type list
        
        #row[0] is the file name .png
        #row[1] is x1
        #row[2] is x2
        #row[3] is y1
        #row[3] is y2
        #row[4] is Frame number
        #print(line_count)
        
        line_count += 1
        
        if(line_count == 1):
            continue           
        
        filename = row[0]
        x1 = float(row[1])
        y1 = float(row[2])
        x2 = float(row[3])
        y2 = float(row[4])
        frame_number = int(row[5])
        
        ed_es_pairing = {}
        if(filename not in my_data):            
            my_data[filename] = ed_es_pairing
        else:
            ed_es_pairing = my_data[filename]
        
        coords = []
        if(frame_number not in ed_es_pairing):            
            ed_es_pairing[frame_number] = coords
        else:
            coords = ed_es_pairing[frame_number]
            
        coords.append((x1,y1,x2,y2))
        
        
        

In [5]:
#sanity check
ed_es_pair = my_data['0X100009310A3BD7FC.avi']
coords = ed_es_pair[61]
print(len(coords))

21


In [6]:
def show_image(img):
    cv2.imshow('hello_world',img)
    
    c = cv2.waitKey()
    if(c>=0):
        return -1
    return 0

def draw_poly_on_image(img, coordinates, color, use_weight=False):
    pts = np.array(coordinates, np.int32)
    pts = pts.reshape((-1,1,2))
    
    weight = 1 
    if (use_weight):
        weight = 2
        
    img = cv2.polylines(img, [pts], True, color, thickness=weight)
    return img

def fill_poly_on_image(img, coordinates, color):
    pts = np.array(coordinates, np.int32)
    pts = pts.reshape((-1,1,2))
    img = cv2.fillPoly(img, [pts], color)
    return img

def draw_line_on_image(img, x1, y1, x2, y2, color, use_weight=False):
    
    weight = 1
    if (use_weight):
        weight = 2
        
    img = cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness=weight)
    return img

In [7]:
#Read video and save Frames for ED and ES

#From https://echonet.github.io/dynamic/index.html  :
#The expert tracings are represented by a collection of paired coordinates 
#corresponding to each human tracing. The first pair of coordinates 
#represent the length and direction of the long axis of the left ventricle, 
#and subsequent coordinate pairs represent short axis linear distances 
#starting from the apex of the heart to the mitral apparatus. Each 
#coordinate pair is also listed with a video file name and frame number to 
#identify the representative frame from which the tracings match.

count = 0
files_that_do_not_exist = []

output_csv_name = os.path.join(outputApproximatedLVCoordinatesPath, outputApproximatedLVCoordinatesPath + '.csv')

with open(output_csv_name, 'w') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Filename', 'Frame_type', 'LV_Coordinates'])    

    for key, value in my_data.items():   

        #count+=1
        #if(count < 5336):
        #    continue

        file_name = key    
        videopath = os.path.join(path_to_videos, file_name)   
        #print(file_name)

        #There are some files mentioned in the csv that do have video files in the echonet data.
        if (not os.path.exists(videopath)):
            print(file_name)
            files_that_do_not_exist.append(file_name)
            continue

        cap = cv2.VideoCapture(videopath)

        cnt = 0
        for frame_number, coords in value.items():        

            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)        
            success, frame = cap.read()       

            #Display the resulting frame
            #show_image(frame)

            # Write the original frame:
            frame_type = 'ED'
            if(cnt==1):
                frame_type = 'ES'
            output_name = file_name + f'_{frame_number}' + f'_{frame_type}' + '.png'
            output_image = os.path.join(outputvolumetracingsFramesPath, output_name)
            cv2.imwrite(output_image, frame)
            cnt += 1

            #Write the original frame with the overlapping expert tracings:
            img = frame.copy()
            coords_list1 = []
            coords_list2 = []

            long_axis = coords[0]
            for coord in coords[1:]:
                x1 = coord[0]
                y1 = coord[1]
                x2 = coord[2]
                y2 = coord[3]
                coords_list1.append((x1, y1))
                coords_list2.append((x2, y2))
                img = draw_line_on_image(img, x1, y1, x2, y2, (0,255,0)) #short axis

            img = draw_line_on_image(img, long_axis[0], long_axis[1], long_axis[2], long_axis[3], (255,0,255))
            output_image1 = os.path.join(outputIMAGESwithExpertTracingsPath, output_name)
            cv2.imwrite(output_image1, img)
            #plt.imshow(img)

            #Write the approximated LV annotation for the original frame
            #LV approximated using the disks defined by short axis
            img = frame.copy()        
            coords_list2.reverse()        
            coords_list3 = coords_list1 + coords_list2
            #Add the apical point which is the first point of the long-axis since we expect the LV to pass
            #through this point
            #We do not need to add the second point of the long axis since this point is the midpoint
            #between the mitral and the bottom tracing line would pass through this point.
            apicalPt = (long_axis[0], long_axis[1])
            #This if-statement is to cater for the bug in the standford code that 
            #builds the excel file volumetracings:
            #for some records, the apical point is actually the second entry. WEIRD.
            if(long_axis[2] < long_axis[1]):
                apicalPt = (long_axis[2], long_axis[3])
            coords_list3.insert(0, apicalPt)
            img = draw_poly_on_image(img, coords_list3, (0,255,0))        
            output_image2 = os.path.join(outputApproximatedLVsPath, output_name)
            cv2.imwrite(output_image2, img)
            #plt.imshow(img)

            #Write the approximated LV binary annotation for the original frame
            blank_image = np.zeros(frame.shape)
            blank = draw_poly_on_image(blank_image, coords_list3, (1,1,1))
            blank = fill_poly_on_image(blank_image, coords_list3, (1,1,1))
            for i in range(frame.shape[0]):
                for j in range(frame.shape[1]):
                    blank[i,j] = np.uint8(blank[i,j])
            output_image3 = os.path.join(outputApproximatedLVBinarPath, output_name)
            cv2.imwrite(output_image3, blank)     
            #plt.imshow(blank)
            #break

            # write a row to the csv file
            writer.writerow([filename, frame_type, coords_list3])

        cap.release()
        #break
    
print('complete')

0X4F8859C8AB4DA9CB.avi
complete


In [8]:
print(count)
print(files_that_do_not_exist)

0
['0X4F8859C8AB4DA9CB.avi']
